In [9]:

import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.signal import spectrogram
import numpy as np
import matplotlib.pyplot as plt
import PIL

In [10]:
def load_emg_column(file_path, column=1):
    data = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            try:
                parts = line.strip().split()
                data.append(float(parts[column]))
            except:
                continue
    return np.array(data)

In [11]:
def window_emg_signal(signal, label, window_size=1000):
    windows = []
    labels = []
    for i in range(0, len(signal) - window_size, window_size):
        windows.append(signal[i:i + window_size])
        labels.append(label)
    return windows, labels

In [12]:
all_windows = []
all_labels = []

for file, label in [
    ('emg_healthy.txt', 0),
    ('emg_myopathy.txt', 1),
    ('emg_neuropathy.txt', 2)
]:
    sig = load_emg_column(file)
    w, l = window_emg_signal(sig, label)
    all_windows.extend(w)
    all_labels.extend(l)

print(f"Total windows: {len(all_windows)}")
print(f"Shape of each window: {all_windows[0].shape}")

Total windows: 307
Shape of each window: (1000,)


In [13]:
def save_emg_as_image(windows, labels, out_dir='emg_images'):
    os.makedirs(out_dir, exist_ok=True)
    for i, (w, label) in enumerate(zip(windows, labels)):
        label_name = {0: "healthy", 1: "myopathy", 2: "neuropathy"}[label]
        class_dir = os.path.join(out_dir, label_name)
        os.makedirs(class_dir, exist_ok=True)

        # Generate spectrogram
        f, t, Sxx = spectrogram(w, fs=1000)  # fs can be changed if known
        plt.figure(figsize=(3, 3))
        plt.pcolormesh(t, f, Sxx, shading='gouraud', cmap='viridis')
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(os.path.join(class_dir, f"{label_name}_{i}.png"), dpi=100, bbox_inches='tight', pad_inches=0)
        plt.close()

In [14]:
save_emg_as_image(all_windows, all_labels)

In [15]:
img_size = (224, 224)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    'emg_images/',
    target_size=img_size,
    batch_size=32,
    class_mode='sparse',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    'emg_images/',
    target_size=img_size,
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

Found 246 images belonging to 3 classes.
Found 61 images belonging to 3 classes.


In [16]:
model = models.Sequential([
    layers.Input(shape=(224,224,3)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_gen, validation_data=val_gen, epochs=10)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.5501 - loss: 2.2412 - val_accuracy: 0.8689 - val_loss: 0.4368
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 949ms/step - accuracy: 0.7844 - loss: 0.4826 - val_accuracy: 0.8689 - val_loss: 0.2196
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.8433 - loss: 0.3199 - val_accuracy: 0.9672 - val_loss: 0.1429
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9604 - loss: 0.1582 - val_accuracy: 0.9672 - val_loss: 0.1141
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.9663 - loss: 0.1016 - val_accuracy: 0.9508 - val_loss: 0.1468
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9942 - loss: 0.0396 - val_accuracy: 0.9180 - val_loss: 0.2008
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9672 - loss: 0.0993 - val_accuracy: 0.9016 - val_loss: 0.2212
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.9118 - loss: 0.2517 - val_accuracy: 0.9344 - val_loss: 0.1525
Epoch 9

In [18]:
model.save('emg_model.keras')

